In [1]:
import pandas as pd
import numpy as np
import sys
import seaborn as sns
import os.path
import time, itertools, re
from collections import Counter
from sklearn.metrics import confusion_matrix, r2_score, root_mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import make_pipeline, Pipeline
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
print(os.getcwd())
fileloc_data='/'.join(os.getcwd().split('/')[0:5])+ '/data/annonymizedDatasets/'
savetag='pred_lavSQ_MHC'
print(fileloc_data)
code_path='/'.join(os.getcwd().split('/')[0:4])+'/sklvq/'
sys.path.append(code_path)
from sklvq import GMLVQ, LGMLVQ
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis,)
from sklearn.svm import SVC, SVR

/home/sreejita/ProjectsPy/GGZ/code_scripts/ED-profiling
/home/sreejita/ProjectsPy/GGZ/data/annonymizedDatasets/


In [2]:
colsLAV=['Lav-Neg_Waardering', 'Lav-Gebrek_Vertrouwdheid', 'Lav-Alg_Ontevredenheid']
colsSQ48=['SQ48-Vijandigheid','SQ48-Agorafobie','SQ48-Angst','SQ48-Depressie', 'SQ48-Cognitieve_Klachten',
          'SQ48-Somatische_Klachten', 'SQ48-Sociale_Fobie','SQ48-Vitaliteit_Optimisme', 'SQ48-Werk_Studie']
colsMHCSF=['MHCSF-EmotionWB', 'MHCSF-SocialWB', 'MHCSF-PsychWB']
cols2consider=['Main-Age','Main-ED_Codes', 'EDEQ-Score']+colsLAV+colsSQ48+colsMHCSF
cols2train=['Main-Age']+colsLAV+colsSQ48+colsMHCSF
colNames=['Main-Age', 'Main-ED_Codes', 'EDEQ-Score','Lav-Neg_Waardering','Lav-Gebrek_Vertrouwdheid', 'Lav-Alg_Ontevredenheid', 
'SQ48-Hostility','SQ48-Agorafobie','SQ48-Angst','SQ48-Depressie','SQ48-Cognitieve_Complain',
'SQ48-Somatische_Complain', 'SQ48-Sociale_Fobie','SQ48-Vitaliteit_Optimisme','SQ48-Werk_Studie', 
'MHCSF-EmotionWB', 'MHCSF-SocialWB', 'MHCSF-PsychWB'] 
colRename_dict=dict(zip(cols2consider, colNames))
df_adapted_combo=pd.read_csv(fileloc_data+'maskedDAIsy_MainED_Lav_SQ48_MHCSF_Visit1.tsv', sep='\t', decimal=',')
df_adapted_combo['EDtype'] = df_adapted_combo['EDtype'].astype('category')
df_adapted_combo.astype(dict(zip(colsLAV, ['float', 'float','float']))).dtypes
df_adapted_combo.astype(dict(zip(colsSQ48,['float', 'float','float','float', 'float','float','float', 'float','float']))).dtypes
df_adapted_combo.astype(dict(zip(colsMHCSF, ['float', 'float','float']))).dtypes
df_adapted_combo.rename(columns=colRename_dict, inplace=True)
adapted_combo_cols=['Main-Age','Lav-Alg_Ontevredenheid', 'Lav-Gebrek_Vertrouwdheid','Lav-Neg_Waardering',
    'SQ48-Hostility','SQ48-Agorafobie','SQ48-Angst','SQ48-Depressie','SQ48-Cognitieve_Complain',
    'SQ48-Somatische_Complain', 'SQ48-Sociale_Fobie','SQ48-Vitaliteit_Optimisme','SQ48-Werk_Studie', 'MHCSF-EmotionWB', 
    'MHCSF-SocialWB', 'MHCSF-PsychWB']
#df_adapted_combo['ED_Codes'] = df_adapted_combo['EDtype'].cat.codes

# Classification

## All classes

In [3]:
df_train_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Train']
YTrain=df_adapted_combo['EDtype'].loc[df_adapted_combo['Split']=='Train'].to_numpy()
df_test_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Test']
YTest=df_adapted_combo['EDtype'].loc[df_adapted_combo['Split']=='Test'].to_numpy()
repeated_kfolds = RepeatedStratifiedKFold(n_splits=3, n_repeats=5)
sampling_strategy='not majority'
sc1=StandardScaler().fit(df_train_adapted)
zX, zXtest=sc1.transform(df_train_adapted), sc1.transform(df_test_adapted)
sm=SMOTE(sampling_strategy=sampling_strategy)
X,Y=sm.fit_resample(zX, YTrain)

In [4]:
iters=5
l_num_prots, g_num_prots=np.array([2, 2, 2, 2, 2]), np.array([2, 2, 2, 2, 2])
pipeRF=Pipeline(steps=[('RF', RandomForestClassifier(criterion="gini", 
                                                            n_estimators=500, max_features=5))])
pipeKNN=Pipeline(steps=[('KNN', KNeighborsClassifier(n_neighbors=3,metric='cosine'))])
pipeLDA=Pipeline(steps=[('LDA', LinearDiscriminantAnalysis(solver='svd'))])
pipeQDA=Pipeline(steps=[('QDA', QuadraticDiscriminantAnalysis())])
pipeLSVC=Pipeline(steps=[('LinSVC', SVC(kernel="linear", C=10))])
pipeRSVC=Pipeline(steps=[('RbfSVC', SVC(kernel='rbf', C=10, gamma=1))])
pipeLGMLVQ=Pipeline(steps=[('LGMLVQ', LGMLVQ(distance_type='local-adaptive-squared-euclidean', prototype_n_per_class=l_num_prots,
        solver_type='lbfgs',activation_type='swish',relevance_n_components=16,relevance_localization="class"))])
pipeGMLVQ=Pipeline(steps=[('GMLVQ', GMLVQ(distance_type='adaptive-squared-euclidean', prototype_n_per_class=g_num_prots,
            activation_type='identity',relevance_n_components=15, solver_type='adam'))])
pipeGNB=Pipeline(steps=[('GNB', GaussianNB())])
pipeClassifiers={'RF':pipeRF, 'KNN':pipeKNN, 'LDA':pipeLDA, 'QDA': pipeQDA, 'LSVC': pipeLSVC, 
                 'RSVC': pipeRSVC, 'LGMLVQ': pipeLGMLVQ, 'GMLVQ': pipeGMLVQ, 'GNB': pipeGNB}
nclasses=len(np.unique(YTrain))
labs=np.unique(YTrain)
train_conf_all, test_conf_all, classifiers_all={},{},{}
        

In [5]:
num_prototypes=np.sum(g_num_prots)
num_features, iters=df_train_adapted.shape[1],5
all_relevance, all_prots=np.zeros((nclasses, num_features, iters)), np.zeros((num_prototypes, num_features, iters))

for key, clf in pipeClassifiers.items():
    tr_cmap=np.zeros((nclasses, nclasses, iters))
    te_cmap=np.zeros((nclasses, nclasses, iters))
    clf_all=[]
    for iter in range(iters):
        if (key=='GMLVQ') | (key=='LGMLVQ'):
            clf.fit(X, Y)
        else:
            clf.fit(X, Y)
        clf_all.append(clf)
        train_pred_labs=clf.predict(zX)
        test_pred_labs=clf.predict(zXtest)
        tr_cmap[:,:,iter]=confusion_matrix(YTrain, train_pred_labs, normalize='true', labels=labs)
        te_cmap[:,:,iter]=confusion_matrix(YTest, test_pred_labs, normalize='true', labels=labs)
    classifiers_all[key]=clf_all
    train_conf_all[key], test_conf_all[key]=tr_cmap, te_cmap
    cmap_dict={'Training': tr_cmap, 'Test':te_cmap}
    for split_set in ['Training', 'Test']:
        print(key+'-'+split_set)    
        if split_set=='Training':
            printlabs=[]
            for lab in labs:
                if (lab==labs[2]) | (lab==labs[4]):
                    printlabs.append('    '+lab)
                else:
                    printlabs.append(lab)
            print(printlabs)
        mean_glob, std_glob=cmap_dict[split_set].mean(axis=2), cmap_dict[split_set].std(axis=2)
        disp_num_glob=np.reshape(np.stack((mean_glob, std_glob)).ravel('F'),(nclasses,2*nclasses))
        #disp_num_glob=[disp_num_glob]
        for idx in range(nclasses):
            cmap_cell=[]
            for jdx in range(nclasses):
                cmap_cell.append('%.3f (%.3f) '%(mean_glob[idx,jdx], std_glob[idx,jdx]))
            print(cmap_cell)    

RF-Training
['Anorexia nervosa', 'Binge-ED', '    Bulimia nervosa', 'Other ED', '    Others']
['1.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ']
['0.000 (0.000) ', '1.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ']
['0.000 (0.000) ', '0.000 (0.000) ', '1.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ']
['0.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '1.000 (0.000) ', '0.000 (0.000) ']
['0.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '1.000 (0.000) ']
RF-Test
['0.508 (0.045) ', '0.077 (0.000) ', '0.069 (0.015) ', '0.162 (0.015) ', '0.185 (0.029) ']
['0.127 (0.045) ', '0.618 (0.068) ', '0.091 (0.000) ', '0.164 (0.036) ', '0.000 (0.000) ']
['0.473 (0.068) ', '0.200 (0.036) ', '0.018 (0.036) ', '0.109 (0.036) ', '0.200 (0.036) ']
['0.071 (0.044) ', '0.224 (0.024) ', '0.071 (0.024) ', '0.365 (0.024) ', '0.271 (0.029) ']
['0.385 (0.000) ', '0.077 (0.000) ', '0.015 (0.031) ', '0.123 (0.038) ', '0.400

## Only ED classes

In [6]:
df_train_eds=df_adapted_combo[adapted_combo_cols].loc[(df_adapted_combo['Split']=='Train') & 
                    (~df_adapted_combo['EDtype'].str.contains('Others'))]
YTrain_eds=df_adapted_combo['EDtype'].loc[(df_adapted_combo['Split']=='Train') & 
                    (~df_adapted_combo['EDtype'].str.contains('Others'))]#.to_numpy()
df_test_eds=df_adapted_combo[adapted_combo_cols].loc[(df_adapted_combo['Split']=='Test') &
                   (~df_adapted_combo['EDtype'].str.contains('Others'))]
YTest_eds=df_adapted_combo['EDtype'].loc[(df_adapted_combo['Split']=='Test') & 
                    (~df_adapted_combo['EDtype'].str.contains('Others'))]#.to_numpy()
sc2=StandardScaler().fit(df_train_eds)
zX_ed, zXtest_ed=sc2.transform(df_train_eds), sc2.transform(df_test_eds)
sm2=SMOTE(sampling_strategy=sampling_strategy)
X_ed,Y_ed=sm2.fit_resample(zX_ed, YTrain_eds)

In [7]:
iters=5
pipeRF2=Pipeline(steps=[('RF', RandomForestClassifier(criterion="gini", 
                                                            n_estimators=250, max_features=11))])
pipeKNN2=Pipeline(steps=[('KNN', KNeighborsClassifier(n_neighbors=3,metric='minkowski'))])
pipeLDA2=Pipeline(steps=[('LDA', LinearDiscriminantAnalysis(solver='svd'))])
pipeQDA2=Pipeline(steps=[('QDA', QuadraticDiscriminantAnalysis())])
pipeLSVC2=Pipeline(steps=[('LinSVC', SVC(kernel="linear", C=100))])
pipeRSVC2=Pipeline(steps=[('RbfSVC', SVC(kernel='rbf', C=10, gamma=1))])
pipeLGMLVQ2=Pipeline(steps=[('LGMLVQ', LGMLVQ(distance_type='local-adaptive-squared-euclidean', 
            prototype_n_per_class=np.array([1, 1, 1,1]),
        activation_type='swish',relevance_n_components=11,relevance_localization="class", solver_type='lbfgs'))])
pipeGMLVQ2=Pipeline(steps=[('GMLVQ', GMLVQ(distance_type='adaptive-squared-euclidean', prototype_n_per_class=np.array([1,1, 1, 1]),
            activation_type='swish',relevance_n_components=11, solver_type='lbfgs'))])
pipeGNB2=Pipeline(steps=[('scaler',  StandardScaler()),('GNB', GaussianNB())])
pipeClassifiers2={'RF':pipeRF2, 'KNN':pipeKNN2, 'LDA':pipeLDA2, 'QDA': pipeQDA2, 'LSVC': pipeLSVC2, 
                 'RSVC': pipeRSVC2, 'LGMLVQ': pipeLGMLVQ2, 'GMLVQ': pipeGMLVQ2, 'GNB': pipeGNB2}
nclasses2=len(np.unique(YTrain_eds))
labs2=np.unique(YTrain_eds)
print(labs2)
train_conf_eds, test_conf_eds, classifiers_eds={},{},{}

['Anorexia nervosa' 'Binge-ED' 'Bulimia nervosa' 'Other ED']


In [8]:
num_prototypes=np.sum(np.array([1,1, 1]))
num_features, iters=df_train_eds.shape[1],5
all_relevance, all_prots=np.zeros((nclasses2, num_features, iters)), np.zeros((num_prototypes, num_features, iters))

for key, clf in pipeClassifiers2.items():
    tr_cmap=np.zeros((nclasses2, nclasses2, iters))
    te_cmap=np.zeros((nclasses2, nclasses2, iters))
    clf_all=[]
    for iter in range(iters):
        clf.fit(X_ed,Y_ed)
        clf_all.append(clf)
        train_pred_labs_eds=clf.predict(zX_ed)
        test_pred_labs_eds=clf.predict(zXtest_ed)
        tr_cmap[:,:,iter]=confusion_matrix(YTrain_eds, train_pred_labs_eds, normalize='true', labels=labs2)
        te_cmap[:,:,iter]=confusion_matrix(YTest_eds, test_pred_labs_eds, normalize='true', labels=labs2)
    classifiers_eds[key]=clf_all
    train_conf_eds[key], test_conf_eds[key]=tr_cmap, te_cmap
    cmap_dict={'Training': tr_cmap, 'Test':te_cmap}
    for split_set in ['Training', 'Test']:
        print(key+'-'+split_set)    
        if split_set=='Training':
            printlabs=[]
            for lab in labs2:
                if (lab==labs2[2]): # | (lab==labs2[3]):
                    printlabs.append('    '+lab)
                else:
                    printlabs.append(lab)
            print(printlabs)
        mean_glob, std_glob=cmap_dict[split_set].mean(axis=2), cmap_dict[split_set].std(axis=2)
        disp_num_glob=np.reshape(np.stack((mean_glob, std_glob)).ravel('F'),(nclasses2,2*nclasses2))
        for idx in range(nclasses2):
            cmap_cell=[]
            for jdx in range(nclasses2):
                cmap_cell.append('%.3f (%.3f) '%(mean_glob[idx,jdx], std_glob[idx,jdx]))
            print(cmap_cell)    

RF-Training
['Anorexia nervosa', 'Binge-ED', '    Bulimia nervosa', 'Other ED']
['1.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ']
['0.000 (0.000) ', '1.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ']
['0.000 (0.000) ', '0.000 (0.000) ', '1.000 (0.000) ', '0.000 (0.000) ']
['0.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '1.000 (0.000) ']
RF-Test
['0.708 (0.031) ', '0.046 (0.015) ', '0.092 (0.046) ', '0.154 (0.034) ']
['0.036 (0.045) ', '0.600 (0.073) ', '0.091 (0.100) ', '0.273 (0.000) ']
['0.582 (0.073) ', '0.218 (0.045) ', '0.091 (0.057) ', '0.109 (0.036) ']
['0.247 (0.044) ', '0.294 (0.000) ', '0.071 (0.024) ', '0.388 (0.060) ']
KNN-Training
['Anorexia nervosa', 'Binge-ED', '    Bulimia nervosa', 'Other ED']
['0.653 (0.000) ', '0.040 (0.000) ', '0.133 (0.000) ', '0.173 (0.000) ']
['0.139 (0.000) ', '0.861 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ']
['0.086 (0.000) ', '0.029 (0.000) ', '0.857 (0.000) ', '0.029 (0.000) ']
['0.143 (0.000) ', '0.024 (0

# Regression
### In regression, the closer the R2 score is to 1 the better is the model's performance, and the closer RMSE is to 0, the better is teh performance

## Regression (all classes)

In [9]:
df_train_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Train']
YTrain_reg=df_adapted_combo['EDEQ-Score'].loc[df_adapted_combo['Split']=='Train']#.to_numpy()
df_test_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Test']
YTest_reg=df_adapted_combo['EDEQ-Score'].loc[df_adapted_combo['Split']=='Test']


In [10]:
iters=5
pipeRFreg=Pipeline(steps=[('RF', RandomForestRegressor(criterion="squared_error", n_estimators=300, max_features=5))])
pipeKNNreg=Pipeline(steps=[('KNN', KNeighborsRegressor(n_neighbors=5,metric='minkowski'))])
pipeLSVR=Pipeline(steps=[('LinSVC', SVR(kernel="linear", C=100))])
pipeRSVR=Pipeline(steps=[('RbfSVC', SVR(kernel='rbf', C=10, gamma=0.01))])
pipeRegressors={'RF':pipeRFreg, 'KNN':pipeKNNreg, 'LSVC': pipeLSVR, 'RSVC': pipeRSVR}

In [11]:
num_features, iters=df_train_eds.shape[1],5
all_relevance=np.zeros((nclasses2, num_features, iters))
perf_dict, regressor_all, train_perf_all, test_perf_all={},{},{},{}
for key, reg in pipeRegressors.items():
    tr_perf=np.zeros((2, iters))
    te_perf=np.zeros((2, iters))
    reg_all=[]
    for iter in range(iters):
        reg.fit(zX, YTrain_reg)
        reg_all.append(reg)
        train_pred_labs_eds=reg.predict(zX)
        test_pred_labs_eds=reg.predict(zXtest)
        tr_perf[0,iter]=r2_score(YTrain_reg, train_pred_labs_eds)
        te_perf[0,iter]=r2_score(YTest_reg, test_pred_labs_eds)
        tr_perf[1,iter]=root_mean_squared_error(YTrain_reg, train_pred_labs_eds)
        te_perf[1,iter]=root_mean_squared_error(YTest_reg, test_pred_labs_eds)
    regressor_all[key]=reg_all
    train_perf_all[key], test_perf_all[key]=tr_perf, te_perf
    perf_dict[key+'Training']={'R2(mean)': np.mean(tr_perf[0,:]),'R2(std)': np.std(tr_perf[0,:]),
                    'RMSE(mean)': np.mean(tr_perf[1,:]),'RMSE(std)': np.std(tr_perf[1,:])}
    perf_dict[key+'Test']={'R2(mean)': np.mean(te_perf[0,:]),'R2(std)': np.std(te_perf[0,:]),
                          'RMSE(mean)': np.mean(te_perf[1,:]),'RMSE(std)': np.std(te_perf[1,:])}
    for split in ['Training', 'Test']:
        print('%s %s: R2= %.3f (%.3f), RMSE= %.3f (%.3f)'%(key, split,
                    perf_dict[key+split]['R2(mean)'], perf_dict[key+split]['R2(std)'],
                    perf_dict[key+split]['RMSE(mean)'], perf_dict[key+split]['RMSE(std)']))
#print(perf_dict)

RF Training: R2= 0.938 (0.002), RMSE= 0.302 (0.005)
RF Test: R2= 0.627 (0.006), RMSE= 0.822 (0.007)
KNN Training: R2= 0.651 (0.000), RMSE= 0.715 (0.000)
KNN Test: R2= 0.617 (0.000), RMSE= 0.832 (0.000)
LSVC Training: R2= 0.590 (0.000), RMSE= 0.775 (0.000)
LSVC Test: R2= 0.610 (0.000), RMSE= 0.840 (0.000)
RSVC Training: R2= 0.720 (0.000), RMSE= 0.641 (0.000)
RSVC Test: R2= 0.621 (0.000), RMSE= 0.828 (0.000)


## Regression (only ED classes)

In [12]:
df_train_eds=df_adapted_combo[adapted_combo_cols].loc[(df_adapted_combo['Split']=='Train') & (df_adapted_combo['EDtype']!='Others')]
YTrain_reg_eds=df_adapted_combo['EDEQ-Score'].loc[(df_adapted_combo['Split']=='Train') & (df_adapted_combo['EDtype']!='Others')]#.to_numpy()
df_test_eds=df_adapted_combo[adapted_combo_cols].loc[(df_adapted_combo['Split']=='Test') & (df_adapted_combo['EDtype']!='Others')]
YTest_reg_eds=df_adapted_combo['EDEQ-Score'].loc[(df_adapted_combo['Split']=='Test') & (df_adapted_combo['EDtype']!='Others')]#.to_numpy()
iters=5
pipeRFreg2=Pipeline(steps=[('RF', RandomForestRegressor(criterion="friedman_mse", n_estimators=100, max_features=5))])
pipeKNNreg2=Pipeline(steps=[('KNN', KNeighborsRegressor(n_neighbors=5,metric='minkowski'))])
pipeLSVR2=Pipeline(steps=[('LinSVC', SVR(kernel="linear", C=0.1))])
pipeRSVR2=Pipeline(steps=[('RbfSVC', SVR(kernel='rbf', C=10, gamma=0.01))])
pipeRegressors2={'RF':pipeRFreg2, 'KNN':pipeKNNreg2, 'LSVC': pipeLSVR2, 'RSVC': pipeRSVR2}

num_features, iters=df_train_eds.shape[1],5
perf_dict2, regressor_all2, train_perf_all2, test_perf_all2={},{},{},{}
for key, reg in pipeRegressors2.items():
    tr_perf=np.zeros((2, iters))
    te_perf=np.zeros((2, iters))
    reg_all=[]
    for iter in range(iters):
        reg.fit(zX_ed, YTrain_reg_eds)
        reg_all.append(reg)
        train_pred_labs_eds=reg.predict(zX_ed)
        test_pred_labs_eds=reg.predict(zXtest_ed)
        tr_perf[0,iter]=r2_score(YTrain_reg_eds, train_pred_labs_eds)
        te_perf[0,iter]=r2_score(YTest_reg_eds, test_pred_labs_eds)
        tr_perf[1,iter]=root_mean_squared_error(YTrain_reg_eds, train_pred_labs_eds)
        te_perf[1,iter]=root_mean_squared_error(YTest_reg_eds, test_pred_labs_eds)
    regressor_all2[key]=reg_all
    train_perf_all2[key], test_perf_all2[key]=tr_perf, te_perf
    perf_dict2[key+'Training']={'R2(mean)': np.mean(tr_perf[0,:]),'R2(std)': np.std(tr_perf[0,:]),
                    'RMSE(mean)': np.mean(tr_perf[1,:]),'RMSE(std)': np.std(tr_perf[1,:])}
    perf_dict2[key+'Test']={'R2(mean)': np.mean(te_perf[0,:]),'R2(std)': np.std(te_perf[0,:]),
                          'RMSE(mean)': np.mean(te_perf[1,:]),'RMSE(std)': np.std(te_perf[1,:])}
    for split in ['Training', 'Test']:
        print('%s %s: R2= %.3f (%.3f), RMSE= %.3f (%.3f)'%(key, split,
                    perf_dict2[key+split]['R2(mean)'], perf_dict2[key+split]['R2(std)'],
                    perf_dict2[key+split]['RMSE(mean)'], perf_dict2[key+split]['RMSE(std)']))


RF Training: R2= 0.934 (0.002), RMSE= 0.318 (0.004)
RF Test: R2= 0.513 (0.023), RMSE= 0.869 (0.020)
KNN Training: R2= 0.617 (0.000), RMSE= 0.766 (0.000)
KNN Test: R2= 0.521 (0.000), RMSE= 0.862 (0.000)
LSVC Training: R2= 0.576 (0.000), RMSE= 0.805 (0.000)
LSVC Test: R2= 0.511 (0.000), RMSE= 0.870 (0.000)
RSVC Training: R2= 0.731 (0.000), RMSE= 0.641 (0.000)
RSVC Test: R2= 0.485 (0.000), RMSE= 0.894 (0.000)


In regression there does not seem to be relevant performance boost by exclusion of the non-ED class patients